In [5]:
import cv2
import numpy as np
import dlib
from imutils import face_utils
import pygame
import time
import math


# ===================== SES =====================
pygame.mixer.init()
pygame.mixer.music.load("alarm.opus")
pygame.mixer.music.set_volume(0.4)

ALARM_INTERVAL = 2.0
last_alarm_time = 0.0

def play_alarm():
    pygame.mixer.music.play()

def stop_alarm():
    pygame.mixer.music.stop()


# ===================== KAMERA & DLIB =====================
cap = cv2.VideoCapture(0)
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")


# ===================== EAR =====================
def euclidean(p1, p2):
    return math.dist(p1, p2)

def eye_aspect_ratio(eye):
    A = euclidean(eye[1], eye[5])
    B = euclidean(eye[2], eye[4])
    C = euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)


# ===================== EŞİKLER (GÜNCEL) =====================
EAR_SLEEP = 0.23
EAR_DROWSY = 0.28
EAR_WAKE = 0.26

SLEEP_TIME = 2.0
DROWSY_TIME = 1.0


eye_closed_start = None
eye_drowsy_start = None

status = "Active :)"
color = (0, 255, 0)


# ===================== ANA DÖNGÜ =====================
while True:
    ret, frame = cap.read()
    if not ret:
        break

    face_frame = frame.copy()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.equalizeHist(gray)

    faces = detector(gray)
    now = time.time()

    for face in faces:
        x1, y1, x2, y2 = face.left(), face.top(), face.right(), face.bottom()
        cv2.rectangle(face_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        landmarks = predictor(gray, face)
        landmarks = face_utils.shape_to_np(landmarks)

        # -------- Landmark çizimi (üst kapak düzeltmeli) --------
        for idx, (x, y) in enumerate(landmarks):
            if idx in [37, 38, 43, 44]:   # üst göz kapağı
                y -= 2                  # SADECE GÖRSEL DÜZELTME
            cv2.circle(face_frame, (x, y), 1, (255, 255, 255), -1)

        left_eye = landmarks[36:42]
        right_eye = landmarks[42:48]

        ear = (eye_aspect_ratio(left_eye) +
               eye_aspect_ratio(right_eye)) / 2.0

        # ===================== SLEEPING =====================
        if ear < EAR_SLEEP:
            if eye_closed_start is None:
                eye_closed_start = now

            if now - eye_closed_start >= SLEEP_TIME:
                status = "SLEEPING !!!"
                color = (0, 0, 255)

                if now - last_alarm_time >= ALARM_INTERVAL:
                    play_alarm()
                    last_alarm_time = now

        else:
            eye_closed_start = None

            if ear > EAR_WAKE:
                stop_alarm()

            # ===================== DROWSY =====================
            if EAR_SLEEP <= ear < EAR_DROWSY:
                if eye_drowsy_start is None:
                    eye_drowsy_start = now

                if now - eye_drowsy_start >= DROWSY_TIME:
                    status = "Drowsy !"
                    color = (0, 255, 255)
            else:
                eye_drowsy_start = None
                status = "Active :)"
                color = (0, 255, 0)

        cv2.putText(
            frame,
            f"{status} | EAR: {ear:.3f}",
            (30, 50),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.9,
            color,
            2
        )

    cv2.imshow("Frame", frame)
    cv2.imshow("Result of detector", face_frame)

    if cv2.waitKey(1) == 27:
        break


cap.release()
cv2.destroyAllWindows()
pygame.mixer.quit()
